In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
# Python Image Library (PIL)

In [3]:
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [4]:
# store every image with its corresponding labels into lists
data = []
labels = []
classes = 43
cur_path = os.getcwd()

# Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        image = Image.open(path + '\\'+ a)
        image = image.resize((30,30))
        image = np.array(image)
        #sim = Image.fromarray(image)
        data.append(image)
        labels.append(i)

In [ ]:
# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

In [ ]:
# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
#Building the model
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [43]:
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(
    monitor = "val_loss",
    min_delta = 0.0001,
    patience = 5,
    verbose = 1,
    mode = "auto",
    baseline = None,
    restore_best_weights = True
)

eps = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=eps, validation_data=(X_test, y_test), callbacks = callback)
# model.save("my_model.h5")

Epoch 1/15
981/981 [==============================] - 71s 70ms/step - loss: 0.3395 - accuracy: 0.9003 - val_loss: 0.1572 - val_accuracy: 0.9481
Epoch 2/15
981/981 [==============================] - 74s 75ms/step - loss: 0.3149 - accuracy: 0.9067 - val_loss: 0.1081 - val_accuracy: 0.9671
Epoch 3/15
981/981 [==============================] - 75s 77ms/step - loss: 0.3070 - accuracy: 0.9093 - val_loss: 0.1200 - val_accuracy: 0.9621
Epoch 4/15
981/981 [==============================] - 70s 72ms/step - loss: 0.3372 - accuracy: 0.9022 - val_loss: 0.0904 - val_accuracy: 0.9717
Epoch 5/15
981/981 [==============================] - 70s 71ms/step - loss: 0.2841 - accuracy: 0.9155 - val_loss: 0.0800 - val_accuracy: 0.9749
Epoch 6/15
981/981 [==============================] - 66s 68ms/step - loss: 0.2725 - accuracy: 0.9197 - val_loss: 0.1181 - val_accuracy: 0.9656
Epoch 7/15
981/981 [==============================] - 69s 70ms/step - loss: 0.2695 - accuracy: 0.9204 - val_loss: 0.0791 - val_accuracy:

In [45]:
model.save("my_model.h5")

In [34]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 64)          36928     
                                                        

In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

# pred = model.predict_classes(X_test)
predict_x = model.predict(X_test)
classes_x = np.argmax(predict_x,axis=1)

In [ ]:
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, classes_x))